In [1]:
import os

out = 'bert-large-bahasa-standard-cased'
os.makedirs(out, exist_ok=True)

In [2]:
directory = out

In [3]:
from transformers import BertTokenizer, BertModel, BertConfig, AutoTokenizer, AutoModelWithLMHead, pipeline

In [4]:
import transformers
transformers.__version__

'3.1.0'

In [5]:
tokenizer = BertTokenizer('BERT.wordpiece', do_lower_case = False)
tokenizer.save_pretrained('bert-large-bahasa-standard-cased')

('bert-large-bahasa-standard-cased/vocab.txt',
 'bert-large-bahasa-standard-cased/special_tokens_map.json',
 'bert-large-bahasa-standard-cased/added_tokens.json')

In [6]:
tokenizer = BertTokenizer.from_pretrained('./bert-large-bahasa-standard-cased', do_lower_case = False)

In [9]:
!transformers-cli convert --model_type bert \
  --tf_checkpoint bert-large/model.ckpt-700000 \
  --config LARGE_config.json \
  --pytorch_dump_output bert-large-bahasa-standard-cased/pytorch_model.bin

Building PyTorch model from configuration: BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 32000
}

Converting TensorFlow checkpoint from /home/husein/bert-standard/bert-large/model.ckpt-700000
Loading TF weight bert/embeddings/LayerNorm/beta with shape [1024]
Loading TF weight bert/embeddings/LayerNorm/beta/adam_m with shape [1024]
Loading TF weight bert/embeddings/LayerNorm/beta/adam_v with shape [1024]
Loading TF weight bert/e

In [11]:
config = BertConfig(f'{directory}/config.json')
config.vocab_size = 32000
config.hidden_size = 1024
config.intermediate_size = 4096
config.num_attention_heads = 16
config.num_hidden_layers = 24

In [12]:
model = AutoModelWithLMHead.from_pretrained('./bert-large-bahasa-standard-cased/pytorch_model.bin', config = config)

/home/husein/.local/lib/python3.6/site-packages/transformers/modeling_auto.py:821: FutureWarning:

The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.

Some weights of the model checkpoint at ./bert-large-bahasa-standard-cased/pytorch_model.bin were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequence

In [13]:
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [14]:
fill_mask('makan ayam dengan [MASK]')

[{'sequence': '[CLS] makan ayam dengan garam [SEP]',
  'score': 0.1847386658191681,
  'token': 12056,
  'token_str': 'garam'},
 {'sequence': '[CLS] makan ayam dengan telur [SEP]',
  'score': 0.06259126961231232,
  'token': 8900,
  'token_str': 'telur'},
 {'sequence': '[CLS] makan ayam dengan daging [SEP]',
  'score': 0.06071213260293007,
  'token': 7630,
  'token_str': 'daging'},
 {'sequence': '[CLS] makan ayam dengan kicap [SEP]',
  'score': 0.037356991320848465,
  'token': 31246,
  'token_str': 'kicap'},
 {'sequence': '[CLS] makan ayam dengan nasi [SEP]',
  'score': 0.03496493399143219,
  'token': 5354,
  'token_str': 'nasi'}]

In [15]:
fill_mask('mahathir sebenarnya sangat [MASK] tanah airnya')

[{'sequence': '[CLS] mahathir sebenarnya sangat mencintai tanah airnya [SEP]',
  'score': 0.29897311329841614,
  'token': 14727,
  'token_str': 'mencintai'},
 {'sequence': '[CLS] mahathir sebenarnya sangat sayangkan tanah airnya [SEP]',
  'score': 0.10250459611415863,
  'token': 22562,
  'token_str': 'sayangkan'},
 {'sequence': '[CLS] mahathir sebenarnya sangat menyayangi tanah airnya [SEP]',
  'score': 0.08254530280828476,
  'token': 27640,
  'token_str': 'menyayangi'},
 {'sequence': '[CLS] mahathir sebenarnya sangat cintakan tanah airnya [SEP]',
  'score': 0.05926104262471199,
  'token': 24174,
  'token_str': 'cintakan'},
 {'sequence': '[CLS] mahathir sebenarnya sangat cantik tanah airnya [SEP]',
  'score': 0.02629738114774227,
  'token': 5190,
  'token_str': 'cantik'}]

In [17]:
model.save_pretrained(out)

In [9]:
!transformers-cli upload ./bert-large-bahasa-standard-cased